In [1]:
import numpy as np
import pandas as pd

import sweetviz as sv

In [2]:
bureau = pd.read_csv('./DATA/bureau.csv')
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [3]:
bureau_balance = pd.read_csv('./DATA/bureau_balance.csv')
bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [4]:
# remove closed/missing balances
invalid_balances = ["X","C"]
bureau_balance = bureau_balance[bureau_balance.eval('STATUS not in @invalid_balances')].copy()

In [5]:
bureau_balance['SK_ID_BUREAU'].value_counts()

5857582    97
6040582    97
5467608    97
6630533    97
6070953    97
           ..
5832530     1
6612002     1
5454618     1
5896580     1
5845073     1
Name: SK_ID_BUREAU, Length: 687027, dtype: int64

In [6]:
id_curr_bureau_map = bureau[['SK_ID_CURR','SK_ID_BUREAU']].copy()
id_curr_bureau_map.head()

,SK_ID_CURR,SK_ID_BUREAU
0,215354,5714462
1,215354,5714463
2,215354,5714464
3,215354,5714465
4,215354,5714466


In [7]:
bureau_balance['DEFAULT'] = False

not_default = ['0']
bureau_balance.loc[bureau_balance.eval('STATUS not in @not_default'), 'DEFAULT'] = True

In [8]:
bureau_balance[bureau_balance['DEFAULT']].head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS,DEFAULT
464,5715793,-39,1,True
465,5715793,-40,1,True
468,5715793,-43,1,True
499,5715795,-30,1,True
500,5715795,-31,1,True


In [9]:
agg_dct = {'DEFAULT':sum,'MONTHS_BALANCE':np.ptp}
bureau_balance_counts = bureau_balance.groupby('SK_ID_BUREAU').agg(agg_dct)
bureau_balance_counts.reset_index(inplace=True)
bureau_balance_counts.head()

,SK_ID_BUREAU,DEFAULT,MONTHS_BALANCE
0,5001710,0,5
1,5001711,0,2
2,5001712,0,9
3,5001716,0,26
4,5001717,0,16


In [10]:
applicant_balances = id_curr_bureau_map.merge(bureau_balance_counts, on='SK_ID_BUREAU', how='left')
applicant_balances.head()

,SK_ID_CURR,SK_ID_BUREAU,DEFAULT,MONTHS_BALANCE
0,215354,5714462,NaN,NaN
1,215354,5714463,NaN,NaN
2,215354,5714464,NaN,NaN
3,215354,5714465,NaN,NaN
4,215354,5714466,NaN,NaN


In [11]:
applicant_balances.fillna(0, inplace=True)

In [12]:
applicant_balances['SK_ID_CURR'].value_counts()

120860    116
169704     94
318065     78
251643     61
425396     60
         ... 
424996      1
227723      1
211331      1
177975      1
274864      1
Name: SK_ID_CURR, Length: 305811, dtype: int64

In [19]:
applicant_default_counts = applicant_balances.groupby('SK_ID_CURR').agg({'DEFAULT':sum,'MONTHS_BALANCE':sum})
applicant_default_counts.reset_index(inplace=True)

In [20]:
col_dct = {'DEFAULT':'DEFAULT_COUNT','MONTHS_BALANCE':'MONTH_COUNT'}
applicant_default_counts.rename(columns=col_dct, inplace=True)

applicant_default_counts.head()

,SK_ID_CURR,DEFAULT_COUNT,MONTH_COUNT
0,100001,1.0,36.0
1,100002,27.0,68.0
2,100003,0.0,0.0
3,100004,0.0,0.0
4,100005,0.0,11.0


In [21]:
report = sv.analyze(applicant_default_counts, pairwise_analysis="off")

                                             |                                             | [  0%]   00:00 ->…

In [22]:
report.show_html('applicant_default_counts.html')

Report applicant_default_counts.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [24]:
applicant_default_counts.to_csv('./DATA/applicant_default_count.csv', index=False)